<a href="https://colab.research.google.com/github/etomaro/RL/blob/main/Grid_world_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


class Agent():

    GAMMA = 0.9  # 割引率
    ACTIONS = ['right', 'up', 'left', 'down']
    # 移動の際に使う値
    act_dict = {
        'right': np.array([0,1]),
        'up': np.array([-1,0]),
        'left': np.array([0,-1]),
        'down': np.array([1,0])
    }
    # 方策(固定)
    pi_dict1 = {
        'right': 0.25,
        'up': 0.25,
        'left': 0.25,
        'down': 0.25
    }
    # 行動数
    num_action = len(ACTIONS)  # 4

    def __init__(self, array_or_list):
        # listもnp.arrayでもいい
        if type(array_or_list) == list:
            array = np.array(array_or_list)
        else:
            array = array_or_list

        # 条件に当てはまらない場合エラーを返す
        assert (array[0] >= 0 and array[0] < 5 and \
                array[1] >= 0 and array[1] < 5)
 
        self.pos = array  # pos -> position


    # 現在位置を返す
    def get_pos(self):
        return self.pos
    
    # 現在位置をセットする
    def set_pos(self, array_or_list):
        # listもnp.arrayでもいい
        if type(array_or_list) == list:
            array = np.array(array_or_list)
        else:
            array = array_or_list

        # 条件に当てはまらない場合エラーを返す
        assert (array[0] >= 0 and array[0] < 5 and \
                array[1] >= 0 and array[1] < 5)
 
        self.pos = array  # pos -> position

    # 現在位置から移動
    def move(self, action):
        """
        - A - B -
        - - - - -
        - - - b -
        - - - - -
        - a - - -
        """
        # 移動量を取得
        move_coord = self.act_dict[action].copy()
    
        # A地点
        if (self.get_pos() == np.array([0,1])).all():
            pos_new = [4,1]  # a地点
        elif (self.get_pos() == np.array([0,3])).all():
            pos_new = [2,3]  # b地点
        else:
            pos_new = self.get_pos() + move_coord 
        
        # グリッドの外に出た場合の処理(出ないようにする)
        pos_new[0] = np.clip(pos_new[0], 0, 4)  # 0未満なら0に。4より上なら4に
        pos_new[1] = np.clip(pos_new[1], 0, 4)
        self.set_pos(pos_new)

    # 現在位置から移動することによる報酬。この関数では移動自体は行わない
    def reward(self, state, action):
        # A地点
        if (state == np.array([0,1])).all():
            r = 10
            return r
        # B地点
        if (state == np.array([0,3])).all():
            r = 5
            return r
        
        # グリッドの外に行く場合は罰則
        if state[0] == 0 and action == 'up':
            r = -1
        elif state[0] == 4 and action == 'down':
            r = -1
        elif state[1] == 0 and action == 'left':
            r = -1
        elif state[1] == 4 and action == 'right':
            r = -1
        else:
            r = 0

        return r
 
    # 方策(π)
    def pi(self, state, action):
        # 変数にstateを持っているが、今回は方策はstateに依存しない
        return self.pi_dict1[action]
    
    # 状態価値関数の算出
    def V_pi(self, state, n, out, iter_num):
        """
        state: 関数呼び出した時の状態
        out: 戻り値用。関数実行時は0を指定。※この引数を消してresult=0としてやった場合変わるかどうかテストする。
        n: 再帰関数の呼び出し回数が何回目か。最初の呼び出しは1
        iter_num: 再帰関数を何回呼び出す予定か。
        """
        if n == iter_num:  # 末端状態
            for action in self.ACTIONS:
                out += self.pi(state, action) * self.reward(state, action)
            return out
        else:
            for action in self.ACTIONS:
                out += self.pi(state, action) * self.reward(state, action)
                # 次の状態に遷移
                self.move(action)

                # 再帰
                out += self.pi(self.get_pos(), action) * \
                       self.V_pi(self.get_pos(), n+1, 0, iter_num) * self.GAMMA

                # stateを関数(または再帰関数)呼び出し時に戻す
                self.set_pos(state)

            return out

    
    # 行動価値関数の算出
    def Q_pi(self):
        pass





In [ ]:
# テスト
# ------Agent------
agent = Agent(np.array([0,0]))

# 1 get_pos()のテスト
now_pos = agent.get_pos()
print("1 get_pos()のテストのテスト\nnow_pos: ", now_pos, "\n")

# 2 set_pos()のテスト
agent.set_pos(np.array([3,3]))
print("2 set_pos()のテスト\nnow_pos: ", agent.get_pos(), "\n")

# 3.1 move()のテスト (3,3) -> (2,3)
agent.move("up")
print("3.1 move()のテスト\nnow_pos: ", agent.get_pos(), "\n")

# 3.2 move()のテスト (4,4) -> right -> (4,4)
agent.set_pos(np.array([4,4]))
agent.move("right")
print("3.2 move()のテスト\nnow_pos: ", agent.get_pos(), "\n")

# 4.1 A地点からアクションした際の報酬と次の状態
agent.set_pos(np.array([0,1]))
reward = agent.reward(agent.get_pos(), "up")
agent.move("up")
print("4.1 A地点からアクションした際の報酬と次の状態\nreward: ", reward, "\nnow_pos: ", agent.get_pos(), "\n")

# 4.2 B地点からアクションした際の報酬と次の状態
agent.set_pos(np.array([0,3]))
reward = agent.reward(agent.get_pos(), "right")
agent.move("right")
print("4.2 B地点からアクションした際の報酬と次の状態\nreward: ", reward, "\nnow_pos: ", agent.get_pos(), "\n")

# 4.3 (4,4)からrightした時の報酬
agent.set_pos(np.array([4,4]))
reward = agent.reward(agent.get_pos(), "right")
agent.move("right")
print("4.3 (4,4)からrightした時の報酬報酬\nreward: ", reward, "\nnow_pos: ", agent.get_pos(), "\n")




1 get_pos()のテストのテスト
now_pos:  [0 0] 

2 set_pos()のテスト
now_pos:  [3 3] 

3.1 move()のテスト
now_pos:  [2 3] 

3.2 move()のテスト
now_pos:  [4 4] 

4.1 A地点からアクションした際の報酬と次の状態
reward:  10 
now_pos:  [4 1] 

4.2 B地点からアクションした際の報酬と次の状態
reward:  5 
now_pos:  [2 3] 

4.3 (4,4)からrightした時の報酬報酬
reward:  -1 
now_pos:  [4 4] 



In [ ]:
# (0,0)地点での状態価値関数の算出
"""
iter=10の時の時
10回行動した時まで考える。
考慮する状態のパターンは4**10(104万8576)
再帰関数を呼ぶ(実行する)回数は回数は9回。関数自体は10回。

計算速度の結果
  iter_num:10 -> 37秒
"""
import time 


agent = Agent(np.array([0,0]))

start = time.time()  # 計測開始
v00 = agent.V_pi(
    state=agent.get_pos(),
    n=1,
    out=0,
    iter_num=10
)
time_result = time.time() - start  # 計測終了

print("v00: ", v00)
print("計測時間: ", time_result)

v00:  3.3383243889638345
計測時間:  37.40410232543945


In [5]:
# fileの作成のテスト

# fileの作成
_filepath = './test.txt'
filecontents = """
1 aiu
2 eo
3 kakiku
6 keko
"""

# 書き込み
# fileが存在しない場合のみ実行する。これがないと同じファイル名で毎度上書きされる
with open(_filepath, 'w') as f:
    f.write(filecontents)

# ファイルの読み込み
with open(_filepath, 'r') as f:
    print(f.read())


1 aiu
2 eo
3 kakiku
5 keko

